In [ ]:
#Importa librerias necesarias
import time
import pandas as pd
from selenium import webdriver
from pymongo import MongoClient

In [ ]:
#Asigna la url de mipleo filtrando los ultimos dias de anuncios
url = "https://www.mipleo.com.ec/ofertas-de-trabajo/?q=&range=15"

In [ ]:
#Conecta con mongodb
Client = MongoClient('localhost',27017)

In [ ]:
#Lista las bases de mongodb
Client.list_database_names()

In [ ]:
#Selecciona la base alldata
db = Client.alldata
db

In [ ]:
#Lista las colecciones
colecciones = db.list_collection_names()
colecciones

In [ ]:
#Selecciona la coleccion creada para mipleo
collection = db["mipleo"]
collection

In [ ]:
#Crea el metodo
def extraer_anuncios(driver):
    print('Realizando scraping...')
    #inicia en 4 por otros elementos de la pagina
    i=4
    try:
        #guarda los anuncios en una lista
        anuncios = driver.find_elements_by_xpath('//*[@id="content"]/div[@class="col_rt big"]/div[@class="item_list"]')
        #recorre la lista
        while i <= len(anuncios):
            #vuelve a guardar la lista de anuncios para actualizarla
            anuncios = driver.find_elements_by_xpath('//*[@id="content"]/div[@class="col_rt big"]/div[@class="item_list"]')
            #selecciona el anuncio deacuerdo a la posicion del while
            anuncio=anuncios[i]
            #convierte la posicion en un string
            a=i
            b=str(a)
            #selecciona el anuncio segun el string y le da click
            driver.find_element_by_xpath('//*[@id="content"]/div[3]/div['+b+']/div/span/a').click()
            #agrega un tiempo de demora
            time.sleep(5)
            #obtiene los datos del empleo
            cargo = driver.find_element_by_xpath('//*[@id="content"]/div[2]/h1').text
            tipocontrato = driver.find_element_by_xpath('//*[@id="content"]/div[3]/div[2]/ul/li[6]/b').text
            ciudad = driver.find_element_by_xpath('//*[@id="content"]/div[3]/div[2]/ul/li[4]/b').text
            fechapublicacion = driver.find_element_by_xpath('//*[@id="content"]/div[3]/div[2]/ul/li[5]/b').text 
            remuneracion = driver.find_element_by_xpath('//*[@id="content"]/div[3]/div[2]/ul/li[1]/b').text
            categoria = driver.find_element_by_xpath('//*[@id="content"]/div[3]/div[2]/ul/li[2]/b').text 
            subcategoria = driver.find_element_by_xpath('//*[@id="content"]/div[3]/div[2]/ul/li[3]/b').text
            contenido = driver.find_element_by_xpath('//*[@id="content"]/div[3]/div[2]/div[2]/p').text.replace('\n',"")
            numcargos = driver.find_element_by_xpath('//*[@id="content"]/div[3]/div[2]/ul/li[8]/b').text
            
            #guarda en un diccionario
            anuncio_dict = {
                'portalweb':"mipleo",
                'cargo':cargo,
                'tipocontrato':tipocontrato,
                'ciudad':ciudad,
                'fechapublicacion':fechapublicacion,
                'remuneracion':remuneracion,
                'categoria':categoria,
                'subcategoria':subcategoria,
                'contenido':contenido,
                'numcargos':numcargos
            }
            
            #consulta si el empleo ya existe
            if collection.find_one(anuncio_dict):
                #si existe regresa
                driver.back()
            else:
                #si no existe inserta en la coleccion
                collection.insert_one(anuncio_dict)
                #regresa
                driver.back()
            #agrega un tiempo de demora
            time.sleep(10)
            i+= 1
    except:
        print('Terminado')

In [ ]:
#Inicia el web driver desde su ubicacion en el equipo
driver = webdriver.Chrome(executable_path='C:/Users/DELL/Downloads/chromedriver.exe')

In [ ]:
#Ingresa a la url
driver.get(url)

In [ ]:
#Ejecuta el metodo
extraer_anuncios(driver)